# Premier modèle K-NN

## Importation des librairies

In [1]:
import pandas as pd
import numpy as np

from create_DataSet import create_DataSet 

## Chargement du DataSet

In [2]:
data = create_DataSet()
data.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Score,Interpretation
0,a,b,c,b,a,1,3.0,2.0,1.0,3.0,10,B
1,a,b,c,a,b,1,3.0,2.0,3.0,1.0,10,B
2,a,b,c,a,b,1,2.0,3.0,1.0,2.0,8,C
3,a,c,a,a,a,2,3.0,2.0,2.0,3.0,10,B
4,a,c,c,b,a,1,NaN,NaN,NaN,3.0,9,C


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215 entries, 0 to 9
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Q1              207 non-null    object
 1   Q2              203 non-null    object
 2   Q3              202 non-null    object
 3   Q4              206 non-null    object
 4   Q5              201 non-null    object
 5   Q6              203 non-null    object
 6   Q7              205 non-null    object
 7   Q8              203 non-null    object
 8   Q9              205 non-null    object
 9   Q10             207 non-null    object
 10  Score           215 non-null    int64 
 11  Interpretation  215 non-null    object
dtypes: int64(1), object(11)
memory usage: 21.8+ KB


In [4]:
print(data.shape)

(215, 12)


## Données erronées

Détermination des données erronées, on cherche donc toutes les valeurs différentes *a*, *b*, *c*, *1*, *2*, *3* .

In [5]:
# Méthode 1 : utilisation de replace avec un regex
data_test = data.copy() 
data_test.iloc[:,:-2] = data_test.iloc[:,:-2].replace(regex=[r'[^a-c1.0-3.0].*'],value='NaN')

In [6]:
data_test.isnull().sum()

Q1                 8
Q2                12
Q3                13
Q4                 9
Q5                14
Q6                12
Q7                10
Q8                12
Q9                10
Q10                8
Score              0
Interpretation     0
dtype: int64

In [7]:
# Methode 2 : utilisation de where et isin
data.iloc[:,:-2] = data.iloc[:,:-2].where(data.iloc[:,:-2].isin(['a','b','c',1,2,3]))

In [8]:
data.isnull().sum()

Q1                 61
Q2                 50
Q3                 51
Q4                 46
Q5                 51
Q6                149
Q7                167
Q8                131
Q9                117
Q10                93
Score               0
Interpretation      0
dtype: int64

## Données manquantes

Comptabilisation du nombre de données manquante.

In [9]:
data.isnull().sum()

Q1                 61
Q2                 50
Q3                 51
Q4                 46
Q5                 51
Q6                149
Q7                167
Q8                131
Q9                117
Q10                93
Score               0
Interpretation      0
dtype: int64

Remplacement des données manquantes par les modes de chaque colonne

In [10]:
data = data.fillna(data.mode().iloc[0,:])
data.isnull().sum()

Q1                0
Q2                0
Q3                0
Q4                0
Q5                0
Q6                0
Q7                0
Q8                0
Q9                0
Q10               0
Score             0
Interpretation    0
dtype: int64

Transformation de toutes les colonnes en _**string**_ afin de pouvoir appliquer _**get_dummies**_ sur toutes les colonnes qualitatives (toutes les colonnes).

In [11]:
print(data.iloc[:,:-2])

   Q1 Q2 Q3 Q4 Q5 Q6   Q7   Q8   Q9  Q10
0   a  b  c  b  a  1  3.0  2.0  1.0  3.0
1   a  b  c  a  b  1  3.0  2.0  3.0  1.0
2   a  b  c  a  b  1  2.0  3.0  1.0  2.0
3   a  c  a  a  a  2  3.0  2.0  2.0  3.0
4   a  c  c  b  a  1  3.0  1.0  1.0  3.0
.. .. .. .. .. .. ..  ...  ...  ...  ...
5   c  c  c  c  c  2  3.0  1.0  1.0    3
6   b  a  c  a  b  2  3.0  1.0  1.0    2
7   a  c  b  a  a  2  3.0  1.0  1.0    3
8   a  b  c  a  b  2  3.0  1.0  1.0    1
9   a  b  c  a  a  2  3.0  1.0  1.0    3

[215 rows x 10 columns]


In [12]:
data.iloc[:,:-2] = data.iloc[:,:-2].replace({1 : 'a', 2 : 'b', 3 : 'c'})

In [13]:
print(data.iloc[:,:-2])

   Q1 Q2 Q3 Q4 Q5 Q6 Q7 Q8 Q9 Q10
0   a  b  c  b  a  a  c  b  a   c
1   a  b  c  a  b  a  c  b  c   a
2   a  b  c  a  b  a  b  c  a   b
3   a  c  a  a  a  b  c  b  b   c
4   a  c  c  b  a  a  c  a  a   c
.. .. .. .. .. .. .. .. .. ..  ..
5   c  c  c  c  c  b  c  a  a   c
6   b  a  c  a  b  b  c  a  a   b
7   a  c  b  a  a  b  c  a  a   c
8   a  b  c  a  b  b  c  a  a   a
9   a  b  c  a  a  b  c  a  a   c

[215 rows x 10 columns]


In [14]:
# data.iloc[:,:-2] = data.iloc[:,:-2].astype(str)

Utilisation de la fonction _**get_dummies**_ pour categoriser les variables qualitatives. 

In [15]:
X = pd.get_dummies(data.iloc[:,:-2]).values

In [16]:
Y = data.iloc[:,-1].values